In [51]:
import pandas as pd 
import geopandas as gpd 
import fiona 
import os
from shapely.geometry import box
import pdal 
import numpy as np
import pyvista as pv

In [4]:
basedir = "/media/riannek/minimax/gleis"
gpkg = "lasfiles_run14_und_run24.gpkg"


gpkg = os.path.join(basedir, gpkg)
fiona.listlayers(gpkg)

['lasfile', 'notes', 'profil', 'layer_styles']

In [8]:
gdf = gpd.read_file(gpkg, layer='lasfile')
gdf.crs 

<Projected CRS: EPSG:5684>
Name: DB_REF / 3-degree Gauss-Kruger zone 4 (E-N)
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Germany - onshore between 10°30'E and 13°30'E.
- bounds: (10.5, 47.39, 13.51, 54.74)
Coordinate Operation:
- name: 3-degree Gauss-Kruger zone 4
- method: Transverse Mercator
Datum: Deutsche Bahn Reference System
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [9]:
gdf.head()

,lasfile_id,filename,filedirectory,messfahrtmission,messfahrttag,messfahrtnr,messfahrtunternr,filesize,filedate,gueltig,geometry
0,135813,2024-08-13/01/run24/01/4473975_5335075.copc.laz,J:/28G/28G0746.200/T3_ID/T3.9_MSE/01_Mobile_Ma...,1,2024-08-13,24,1,38399274,2024-11-29 22:55:09.521,True,"POLYGON ((4474000 5335075, 4474000 5335100, 44..."
1,135814,2024-08-13/01/run24/01/4473975_5335100.copc.laz,J:/28G/28G0746.200/T3_ID/T3.9_MSE/01_Mobile_Ma...,1,2024-08-13,24,1,34373761,2024-11-29 22:55:32.865,True,"POLYGON ((4474000 5335100, 4474000 5335125, 44..."
2,135815,2024-08-13/01/run24/01/4473975_5335125.copc.laz,J:/28G/28G0746.200/T3_ID/T3.9_MSE/01_Mobile_Ma...,1,2024-08-13,24,1,24133357,2024-11-29 22:55:54.381,True,"POLYGON ((4474000 5335125, 4474000 5335150, 44..."
3,135816,2024-08-13/01/run24/01/4473975_5335150.copc.laz,J:/28G/28G0746.200/T3_ID/T3.9_MSE/01_Mobile_Ma...,1,2024-08-13,24,1,18793899,2024-11-29 22:56:07.616,True,"POLYGON ((4474000 5335150, 4474000 5335175, 44..."
4,135817,2024-08-13/01/run24/01/4473975_5335175.copc.laz,J:/28G/28G0746.200/T3_ID/T3.9_MSE/01_Mobile_Ma...,1,2024-08-13,24,1,13118441,2024-11-29 22:56:26.013,True,"POLYGON ((4474000 5335175, 4474000 5335200, 44..."


In [11]:
run = gdf[gdf["messfahrtnr"] == 24]

In [20]:
bounds = run.bounds 
bounds.head()

,minx,miny,maxx,maxy
0,4473975.0,5335075.0,4474000.0,5335100.0
1,4473975.0,5335100.0,4474000.0,5335125.0
2,4473975.0,5335125.0,4474000.0,5335150.0
3,4473975.0,5335150.0,4474000.0,5335175.0
4,4473975.0,5335175.0,4474000.0,5335200.0


In [33]:
margin = 2

In [21]:
bounds["minx"] -= 2 
bounds["miny"] -= 2 
bounds["maxx"] += 2
bounds["maxy"] += 2

bounds.head()

,minx,miny,maxx,maxy
0,4473973.0,5335073.0,4474002.0,5335102.0
1,4473973.0,5335098.0,4474002.0,5335127.0
2,4473973.0,5335123.0,4474002.0,5335152.0
3,4473973.0,5335148.0,4474002.0,5335177.0
4,4473973.0,5335173.0,4474002.0,5335202.0


In [46]:
bounds.loc[0]

minx    4473973.0
miny    5335073.0
maxx    4474002.0
maxy    5335102.0
Name: 0, dtype: float64

In [23]:
with_margin = gpd.GeoDataFrame(geometry=[box(minx, miny, maxx, maxy) for minx, miny, maxx, maxy in bounds.values], crs=gdf.crs)

In [24]:
with_margin.head()

,geometry
0,"POLYGON ((4474002 5335073, 4474002 5335102, 44..."
1,"POLYGON ((4474002 5335098, 4474002 5335127, 44..."
2,"POLYGON ((4474002 5335123, 4474002 5335152, 44..."
3,"POLYGON ((4474002 5335148, 4474002 5335177, 44..."
4,"POLYGON ((4474002 5335173, 4474002 5335202, 44..."


In [36]:
expanded_bounds = with_margin.loc[0].geometry

In [77]:
bbox = bounds.loc[0]

In [44]:
run[run.intersects(expanded_bounds)]["filename"]

0       2024-08-13/01/run24/01/4473975_5335075.copc.laz
1       2024-08-13/01/run24/01/4473975_5335100.copc.laz
59      2024-08-13/01/run24/01/4474000_5335050.copc.laz
60      2024-08-13/01/run24/01/4474000_5335075.copc.laz
61      2024-08-13/01/run24/01/4474000_5335100.copc.laz
8113    2024-08-13/01/run24/01/4473950_5335050.copc.laz
8114    2024-08-13/01/run24/01/4473950_5335075.copc.laz
8115    2024-08-13/01/run24/01/4473950_5335100.copc.laz
8175    2024-08-13/01/run24/01/4473975_5335050.copc.laz
Name: filename, dtype: object

In [50]:
files = run[run.intersects(expanded_bounds)]["filename"].to_list() 

In [53]:
filename = os.path.join(basedir, files[0])

pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

In [65]:
pipeline = pdal.Reader(os.path.join(basedir, files[0])) | pdal.Reader(os.path.join(basedir, files[1]))
pipeline.execute()
points = pipeline.arrays[0]

In [67]:
pipeline = pdal.Pipeline([pdal.Reader(os.path.join(basedir, files[0])), pdal.Reader(os.path.join(basedir, files[1]))])
pipeline.execute()
points = pipeline.arrays[0]

In [78]:
bbox

minx    4473973.0
miny    5335073.0
maxx    4474002.0
maxy    5335102.0
Name: 0, dtype: float64

In [81]:
b = f"([{bbox.minx},{bbox.miny}], [{bbox.maxx},{bbox.maxy}])"
b

'([4473973.0,5335073.0], [4474002.0,5335102.0])'

- Jede Kachel kann 8 Nachbarn haben, macht 9 Dateien.
- Das Laden dauert dann 10 s!

In [82]:
pipeline = pdal.Pipeline([pdal.Reader(os.path.join(basedir, file)) for file in files]) | pdal.Filter.merge() | pdal.Filter.crop(bounds=b)
pipeline.execute()
points = pipeline.arrays[0]

In [84]:
xyz = np.column_stack((points['X'], points['Y'], points['Z']))
pcd = pv.PolyData(xyz)
pcd['Intensity'] = points['Intensity']
pcd.plot(scalars="Intensity")

Widget(value='<iframe src="http://localhost:33913/index.html?ui=P_0x7fcd842e85b0_5&reconnect=auto" class="pyvi…